In [1]:
print('hi')

hi


In [2]:
from app.llm_client import OpenAIClient
from app.agent import AIAgent
from app.tools import retrieve_docs
from app.memory import SessionMemory


In [3]:
# Setup
llm = OpenAIClient()
memory = SessionMemory()
tools = {"retrieve_docs": retrieve_docs}
agent = AIAgent(llm, tools, memory)

In [8]:
result = agent.decide_and_answer("how many days a person can take leave in a year", session_id="abc123")
print(result)

{'answer': 'Employees are entitled to 20 paid leave days per year.', 'sources': ['leave_policy.txt', 'remote_work_policy.txt']}


In [7]:
print(result['answer'])

In the context of large language models (LLMs), "hallucination" refers to the phenomenon where the model generates information that is false, misleading, or not grounded in reality. This can occur when the model produces responses that sound plausible but are actually incorrect or fabricated. Hallucinations can arise due to various reasons, including:

1. **Training Data Limitations**: The model may not have been trained on accurate or comprehensive data regarding a specific topic.
2. **Inference Errors**: During the generation process, the model might make incorrect associations or extrapolations based on its training.
3. **Ambiguity in Queries**: Vague or ambiguous questions can lead the model to generate responses that do not accurately reflect the intended meaning.

Hallucinations are a known challenge in the deployment of LLMs and are an area of active research to improve the reliability and accuracy of these models.


In [ ]:
# #run only once
# from app.rag import create_faiss_index
# create_faiss_index()


In [9]:
a={'a':2324,
   'b':563}

In [12]:
a.get('c', [])

[]

In [1]:
import faiss
import pickle

INDEX_PATH = "embeddings/faiss.index"
DOCS_PATH = "embeddings/docs.pkl"
index = faiss.read_index(INDEX_PATH)

with open(DOCS_PATH, "rb") as f:
    docs = pickle.load(f)

In [5]:
docs

[{'text': 'Employees are entitled to 20 paid leave days per year.\nLeave requests must be submitted at least 5 working days in advance.\nUnused leave cannot be carried forward to the next year.\n',
  'source': 'leave_policy.txt'},
 {'text': 'Employees may work remotely up to 3 days per week.\nRemote work requires manager approval.\nEmployees must be available during core working hours.\n',
  'source': 'remote_work_policy.txt'},
 {'text': 'All employees must use company-approved devices.\nPasswords must be changed every 90 days.\nSharing credentials is strictly prohibited.\n',
  'source': 'security_policy.txt'}]

In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np
load_dotenv()
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

EMBEDDING_MODEL = "text-embedding-3-small"

In [4]:
query='tell me more about leave policy'

In [6]:
query_embedding = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=[query]
    ).data[0].embedding

In [8]:
distances, indices = index.search(
        np.array([query_embedding]).astype("float32"), 2
    )

In [10]:
indices

array([[0, 1]])

In [ ]:
def search(query, k=2):
    index = faiss.read_index(INDEX_PATH)

    with open(DOCS_PATH, "rb") as f:
        docs = pickle.load(f)

    query_embedding = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=[query]
    ).data[0].embedding

    distances, indices = index.search(
        np.array([query_embedding]).astype("float32"), k
    )

    results = []
    for i in indices[0]:
        results.append(docs[i])

    return results

In [1]:
import os
import faiss
import pickle
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

EMBEDDING_MODEL = "text-embedding-3-small"

DOCS_PATH = "data/documents"
EMBED_DIR = "embeddings_new"
INDEX_PATH = f"{EMBED_DIR}/faiss.index"
DOCS_PKL = f"{EMBED_DIR}/docs.pkl"

In [26]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader(
    path="data/documents",              # folder containing PDFs
    glob="**/*.pdf",           # load all PDFs recursively
    loader_cls=PyPDFLoader
)

documents = loader.load()

print(f"Loaded {len(documents)} pages")


Loaded 12 pages


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

docs = text_splitter.split_documents(documents)

print(f"Total chunks: {len(docs)}")

Total chunks: 40


In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

vectorstore = FAISS.from_documents(docs, embeddings)

In [20]:
results = vectorstore.similarity_search('how many days a user can take leave in a year from leave policy', k=3)

In [21]:
results

[Document(id='428c132d-3f17-4357-a936-479dddde6bdb', metadata={'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': 'data\\documents\\security_policy.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='employment.\n \n \n4.  Leave  Accrual  and  Usage  \n●  Leave  days  must  be  used  within  the  same  calendar  year  in  which  they  are  granted.  ●  Unused  leave  cannot  be  carried  forward  to  the  next  year.  ●  Any  remaining  leave  balance  at  the  end  of  the  year  will  automatically  lapse.  ●  Leave  encashment  is  not  permitted  unless  explicitly  stated  in  the  employment  \ncontract.'),
 Document(id='fd40e921-2fb3-4d09-b1a1-1087363faddd', metadata={'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': 'data\\documents\\security_policy.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, p

In [22]:
contexts = []
sources = []

for doc in results:
    contexts.append(doc.page_content)
    sources.append({
        "document": doc.metadata.get("source"),
        "page": doc.metadata.get("page") + 1  # human-readable
    })

In [23]:
sources

[{'document': 'data\\documents\\security_policy.pdf', 'page': 1},
 {'document': 'data\\documents\\security_policy.pdf', 'page': 1},
 {'document': 'data\\documents\\security_policy.pdf', 'page': 1}]

In [24]:
contexts

['employment.\n \n \n4.  Leave  Accrual  and  Usage  \n●  Leave  days  must  be  used  within  the  same  calendar  year  in  which  they  are  granted.  ●  Unused  leave  cannot  be  carried  forward  to  the  next  year.  ●  Any  remaining  leave  balance  at  the  end  of  the  year  will  automatically  lapse.  ●  Leave  encashment  is  not  permitted  unless  explicitly  stated  in  the  employment  \ncontract.',
 'with\n \nadequate\n \ntime\n \nfor\n \nrest,\n \npersonal\n \nmatters,\n \nand\n \nwork-life\n \nbalance.\n \n●  Leave  entitlement  is  granted  on  a  yearly  basis.  ●  Employees  who  join  mid-year  will  receive  leave  on  a  pro-rated  basis.  ●  Paid  leave  days  are  credited  at  the  beginning  of  the  year  or  upon  confirmation  of  \nemployment.\n \n \n4.  Leave  Accrual  and  Usage',
 'in\n \ntheir\n \nrespective\n \nagreements.\n \n \n3.  Leave  Entitlement  \n3.1  Annual  Paid  Leave  \nEmployees  are  entitled  to  20  paid  leave  days  per  calen

In [1]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

load_dotenv()

DOCS_PATH = "data/documents"
EMBED_DIR = "embeddings_new"
INDEX_PATH = os.path.join(EMBED_DIR, "faiss.index")

EMBEDDING_MODEL = "text-embedding-3-large"

In [10]:



def build_faiss():
    print("🔧 Building FAISS index...")

    loader = DirectoryLoader(
        path=DOCS_PATH,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    print(f"📄 Loaded {len(documents)} pages")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )

    docs = text_splitter.split_documents(documents)
    print(f"✂️ Total chunks: {len(docs)}")

    embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

    vectorstore = FAISS.from_documents(docs, embeddings)

    os.makedirs(EMBED_DIR, exist_ok=True)
    vectorstore.save_local(EMBED_DIR)

    print("✅ FAISS built and saved successfully")


def load_or_build_faiss():
    embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

    if os.path.exists(EMBED_DIR):
        print("📦 Loading existing FAISS index...")
        return FAISS.load_local(
            EMBED_DIR,
            embeddings,
            allow_dangerous_deserialization=True
        )
    else:
        build_faiss()
        return FAISS.load_local(
            EMBED_DIR,
            embeddings,
            allow_dangerous_deserialization=True
        )




In [11]:
INDEX_PATH

'embeddings_new\\faiss.index'

In [12]:

# ✅ Usage
vectorstore = load_or_build_faiss()

📦 Loading existing FAISS index...


In [13]:
results = vectorstore.similarity_search('how many days a user can take leave in a year from leave policy', k=3)

In [16]:
results

[Document(id='9860190e-f783-4b6d-9527-cf7f2f7556c4', metadata={'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': 'data\\documents\\leave_policy.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='employment.\n \n \n4.  Leave  Accrual  and  Usage  \n●  Leave  days  must  be  used  within  the  same  calendar  year  in  which  they  are  granted.  ●  Unused  leave  cannot  be  carried  forward  to  the  next  year.  ●  Any  remaining  leave  balance  at  the  end  of  the  year  will  automatically  lapse.  ●  Leave  encashment  is  not  permitted  unless  explicitly  stated  in  the  employment  \ncontract.'),
 Document(id='ddffe0b5-5bee-496d-b30a-d9be77a21f72', metadata={'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': 'data\\documents\\leave_policy.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_co

In [17]:
contexts = []
sources = []

for doc in results:
    contexts.append(doc.page_content)
    sources.append({
        "document": doc.metadata.get("source"),
        "page": doc.metadata.get("page") + 1  # human-readable
    })

In [18]:
contexts

['employment.\n \n \n4.  Leave  Accrual  and  Usage  \n●  Leave  days  must  be  used  within  the  same  calendar  year  in  which  they  are  granted.  ●  Unused  leave  cannot  be  carried  forward  to  the  next  year.  ●  Any  remaining  leave  balance  at  the  end  of  the  year  will  automatically  lapse.  ●  Leave  encashment  is  not  permitted  unless  explicitly  stated  in  the  employment  \ncontract.',
 'with\n \nadequate\n \ntime\n \nfor\n \nrest,\n \npersonal\n \nmatters,\n \nand\n \nwork-life\n \nbalance.\n \n●  Leave  entitlement  is  granted  on  a  yearly  basis.  ●  Employees  who  join  mid-year  will  receive  leave  on  a  pro-rated  basis.  ●  Paid  leave  days  are  credited  at  the  beginning  of  the  year  or  upon  confirmation  of  \nemployment.\n \n \n4.  Leave  Accrual  and  Usage',
 'in\n \ntheir\n \nrespective\n \nagreements.\n \n \n3.  Leave  Entitlement  \n3.1  Annual  Paid  Leave  \nEmployees  are  entitled  to  20  paid  leave  days  per  calen

In [19]:
sources

[{'document': 'data\\documents\\leave_policy.pdf', 'page': 1},
 {'document': 'data\\documents\\leave_policy.pdf', 'page': 1},
 {'document': 'data\\documents\\leave_policy.pdf', 'page': 1}]

In [28]:
for i in sources:
    print(i['document'].split('\\')[-1])
    break

leave_policy.pdf


In [32]:
list(set([i['document'].split('\\')[-1] for i in sources]))

['leave_policy.pdf']

In [38]:
[i.page_content for i in results]


['employment.\n \n \n4.  Leave  Accrual  and  Usage  \n●  Leave  days  must  be  used  within  the  same  calendar  year  in  which  they  are  granted.  ●  Unused  leave  cannot  be  carried  forward  to  the  next  year.  ●  Any  remaining  leave  balance  at  the  end  of  the  year  will  automatically  lapse.  ●  Leave  encashment  is  not  permitted  unless  explicitly  stated  in  the  employment  \ncontract.',
 'with\n \nadequate\n \ntime\n \nfor\n \nrest,\n \npersonal\n \nmatters,\n \nand\n \nwork-life\n \nbalance.\n \n●  Leave  entitlement  is  granted  on  a  yearly  basis.  ●  Employees  who  join  mid-year  will  receive  leave  on  a  pro-rated  basis.  ●  Paid  leave  days  are  credited  at  the  beginning  of  the  year  or  upon  confirmation  of  \nemployment.\n \n \n4.  Leave  Accrual  and  Usage',
 'in\n \ntheir\n \nrespective\n \nagreements.\n \n \n3.  Leave  Entitlement  \n3.1  Annual  Paid  Leave  \nEmployees  are  entitled  to  20  paid  leave  days  per  calen

In [ ]:
for i in results:
    print(i.metadata.get("source").spli)
    break

data\documents\leave_policy.pdf


In [42]:
from app.prompts import AGENT_SYSTEM_PROMPT

In [43]:
from fastapi import FastAPI
from app.schemas import AskRequest, AskResponse
from app.agent import AIAgent
from app.memory import SessionMemory
from app.tools import retrieve_docs
from app.llm_client import OpenAIClient

In [47]:


# Initialize the client
client = OpenAIClient()

# Prepare messages (like the chat format OpenAI expects)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]

# Get the response
answer = client.chat(messages)

In [48]:
answer

'The capital of France is Paris.'

In [46]:
ab.chat('hi')

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages': expected an array of objects, but got a string instead.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'invalid_type'}}

In [ ]:
chat(answer_messages)